# Harrow–Hassidim–Loyd algorithm(HHL) algoritum demo using Q-alchemy's state preparation

HHL algorithm is a quantum algorithm that solves a systems of linear equations. In general, a system of linear $n$-dimensional equations  can be written as follows.

$$
A \vec{x} = \vec{b}
$$

Where $A$ is $n$ x $n$ matrix and $\vec{x}, \vec{b} \in \mathbb{R}^n$. HHL algorithm is proven to provide an exponential speed up given that the $A$ is a sparse, matrix and the condition number $\kappa$ of the system is low.  The condition number $\kappa$ is a measure of how much the output value of the function can change for a small change in the input argument. 

To run the HHL algorithm we first have to map the solution vector $\vec{b}$ to the Hilber space. In this module, we will present how the state preparation algorithm by Q-alchemy can be implemented in this context, and compare the results with IBM's state preparation method.


In [8]:
from qiskit import transpile
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from scipy.sparse import diags

from linear_solvers import NumPyLinearSolver, HHL
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.quantum_info import Statevector
import sys
import os
os.environ["Q_ALCHEMY_API_KEY"] = "JnvkpMCsyr4nB9nHcwa6CbxqhtZXyF1b"
sys.path.append('..')
from q_alchemy.qiskit import QAlchemyInitialize
from qiskit.opflow import (
    Z,
    I,
    StateFn,
    TensoredOp,
    ExpectationBase,
    CircuitSampler,
    ListOp,
    ExpectationFactory,
    ComposedOp,
)



['/Users/wooseophwang/Desktop/q-alchemy-sdk-py/examples', '/Users/wooseophwang/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles', '/Users/wooseophwang/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles/lib/python', '/Users/wooseophwang/anaconda3/envs/new/lib/python311.zip', '/Users/wooseophwang/anaconda3/envs/new/lib/python3.11', '/Users/wooseophwang/anaconda3/envs/new/lib/python3.11/lib-dynload', '', '/Users/wooseophwang/anaconda3/envs/new/lib/python3.11/site-packages', './', './', './', './', '..', '..']


In [ ]:
def calculate_norm(qc: QuantumCircuit, nb: int, nl: int, na:int) -> float:
        """Calculates the value of the euclidean norm of the solution.

        Args:
            qc: The quantum circuit preparing the solution x to the system.

        Returns:
            The value of the euclidean norm of the solution.
        """

        # Create the Operators Zero and One
        zero_op = (I + Z) / 2
        one_op = (I - Z) / 2

        # Norm observable
        observable = one_op ^ TensoredOp((nl + na) * [zero_op]) ^ (I ^ nb)
        norm_2 = (~StateFn(observable) @ StateFn(qc)).eval()
        

        return np.real(np.sqrt(norm_2))

In [ ]:
def get_solution_vector(solution, n):
    solution_vector = Statevector(solution.state).data[n:n+8].real
    norm = solution.euclidean_norm
    return norm * solution_vector / np.linalg.norm(solution_vector)

## Define matrix $A$ as 8 by 8 Sparse Hermitian matrice
First, we will begin by defining the matrix $A$, and $b$. We will arbitarily define A, and randomly generate corresponding solution vector $\vec{b}$ and normalize it.

In [ ]:
A = np.array([[1.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [2.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [3.0, 4.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 5.0, 6.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 10.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 12.0]])

# Ensure Hermitian property (conjugate transpose)
A = (A + A.T) / 2.0
#b = [1, 0, 0, 0, 0, 0, 0, 0]
b = np.random.rand(8)
b = b/np.linalg.norm(b) #normalize solution

### Initialize the solution state using Q-alchemy

From $\vec{b}$ we have generated above, we will initialize Q-alchemy and generate a qunutum circuit that transforms the quantum state from $\ket{0}$ to $\ket{b}$, where $\ket{b} = \sum_{i=1}^{n} b_i \ket{i}$.

In [ ]:
sp_org = QAlchemyInitialize(b, opt_params={'max_fidelity_loss':0.0})
sp_org.definition.draw(fold=-1)

┌──────────────────────────────────────────────────────────────────────┐
q_0: ┤0                                                                     ├
     │                                                                      │
q_1: ┤1 LRSP(0.41153,0.12419,0.429,0.40731,0.53583,0.26869,0.21978,0.24014) ├
     │                                                                      │
q_2: ┤2                                                                     ├
     └──────────────────────────────────────────────────────────────────────┘

We compare the state preapred by the Q-alchemy, and the actual target state $\vec{b}$.

In [ ]:
qc = transpile(sp_org.definition, basis_gates=["id", "rx", "ry", "rz", "cx"])
num_q1 = qc.num_qubits
print('state prepared by Q-alchemy',Statevector(qc).data.real)
print('Target state',b)
qc.draw()

state prepared by Q-alchemy [0.41152717 0.12418922 0.42900313 0.40730592 0.53582653 0.26869457
 0.21978396 0.2401433 ]
Target state [0.41152717 0.12418922 0.42900313 0.40730592 0.53582653 0.26869457
 0.21978396 0.2401433 ]


global phase: 2π
     ┌─────────────┐           ┌─────────────┐ ┌────────┐                »
q_0: ┤ Ry(0.38787) ├─────■─────┤ Ry(-1.1506) ├─┤ Rz(-π) ├────────────────»
     └─────────────┘   ┌─┴─┐   ├─────────────┴┐└────────┘┌──────────────┐»
q_1: ──────────────────┤ X ├───┤ Ry(-0.76032) ├────■─────┤ Rx(-0.30914) ├»
     ┌─────────────┐┌──┴───┴──┐└──────────────┘  ┌─┴─┐   ├──────────────┤»
q_2: ┤ Ry(0.55153) ├┤ Rz(π/2) ├──────────────────┤ X ├───┤ Rz(-0.61548) ├»
     └─────────────┘└─────────┘                  └───┘   └──────────────┘»
«                                                                    »
«q_0: ───────────────────────────────────────────────────────────────»
«      ┌────────┐                      ┌─────────┐  ┌───────────────┐»
«q_1: ─┤ Rz(-π) ├──────────────────■───┤ Rz(π/2) ├──┤ Ry(0.0020268) ├»
«     ┌┴────────┤┌──────────────┐┌─┴─┐┌┴─────────┴─┐└──┬──────────┬─┘»
«q_2: ┤ Ry(π/3) ├┤ Rz(-0.61548) ├┤ X ├┤ Rz(2.5261) ├───┤ Ry(2π/3) ├──»
«     └─────────┘└──────────────┘└───┘└────────────┘   └──────────┘  »
«                                                   
«q_0: ──────────────────────────────────────────────
«                         ┌─────────┐┌─────────────┐
«q_1: ─────────────────■──┤ Rz(π/2) ├┤ Ry(0.77913) ├
«     ┌─────────────┐┌─┴─┐├─────────┤├─────────────┤
«q_2: ┤ Rz(-2.5261) ├┤ X ├┤ Rz(π/2) ├┤ Ry(0.53453) ├
«     └─────────────┘└───┘└─────────┘└─────────────┘

### Use IBM's module to construct circuit that performs HHL algorithm

Then, we pass the state preparation circuit created by Q-alchemy to IBM's HHL module, so we can perform the rest of HHL algorithm based on the solution state created by Q-alchemy.

In [ ]:
qal_hhl = HHL().solve(A, qc)

num_q = qal_hhl.state.num_qubits
qal_hhl.state.draw()

┌──────────────┐┌───────┐        ┌──────────┐
q12_0: ┤0             ├┤9      ├────────┤9         ├
       │              ││       │        │          │
q12_1: ┤1 circuit-127 ├┤10     ├────────┤10        ├
       │              ││       │        │          │
q12_2: ┤2             ├┤11     ├────────┤11        ├
       └──────────────┘│       │┌──────┐│          │
q13_0: ────────────────┤0      ├┤8     ├┤0         ├
                       │       ││      ││          │
q13_1: ────────────────┤1      ├┤7     ├┤1         ├
                       │       ││      ││          │
q13_2: ────────────────┤2      ├┤6     ├┤2         ├
                       │   QPE ││      ││   QPE_dg │
q13_3: ────────────────┤3      ├┤5     ├┤3         ├
                       │       ││      ││          │
q13_4: ────────────────┤4      ├┤4     ├┤4         ├
                       │       ││  1/x ││          │
q13_5: ────────────────┤5      ├┤3     ├┤5         ├
                       │       ││      ││          │
q13_6: ────────────────┤6      ├┤2     ├┤6         ├
                       │       ││      ││          │
q13_7: ────────────────┤7      ├┤1     ├┤7         ├
                       │       ││      ││          │
q13_8: ────────────────┤8      ├┤0     ├┤8         ├
                       └───────┘│      │└──────────┘
  q14: ─────────────────────────┤9     ├────────────
                                └──────┘

### Comparison of Q-alchemy's solution with IBM's default state preparation method, and classical solution

In this section, we are going to compare the $\vec{x}$ obtained based on Q-alchemy's state, and that obtained based on IBM's default state preparation method and the classical solution.

In [ ]:
classical_solution = NumPyLinearSolver().solve(A, b)
hhl_default = HHL().solve(A, b) #using IBM's default state preparation method

x0 = 2**(num_q-1)

qal_sol = get_solution_vector(qal_hhl, x0)
default_sol = get_solution_vector(hhl_default, x0)
classic_sol = classical_solution.state

print('solution with Q-alchemy:', qal_sol)
print('solution with IBM:', default_sol)
print('classical solution:', classic_sol)



solution with Q-alchemy: [-1.03523645e+00  7.96568707e-01 -4.88290296e-02  1.08494218e-01
  7.75745370e-02 -6.56233835e-04  6.01811561e-02 -3.46821168e-02]
solution with IBM: [-1.03523645e+00  7.96568707e-01 -4.88290296e-02  1.08494218e-01
  7.75745370e-02 -6.56233835e-04  6.01811561e-02 -3.46821168e-02]
classical solution: [-1.03511861e+00  7.96696195e-01 -4.89155357e-02  1.08647266e-01
  7.71245412e-02 -5.05656973e-04  5.97909899e-02 -3.43750852e-02]


In [ ]:
print('Q-alchemy euclidean norm:', qal_hhl.euclidean_norm)
print('IBM euclidean norm:', hhl_default.euclidean_norm)
print('Classical euclidean norm:', classical_solution.euclidean_norm)


Q-alchemy euclidean norm: 1.3157637838533367
IBM euclidean norm: 1.3157637838533354
Classical euclidean norm: 1.3157117421522682


### Compare the state prepared by Q alchemy and IBM

In [ ]:
hhl_default.state.data.pop(-1)
state_IBM = Statevector(hhl_default.state).data.real
hhl_default.state.draw()



┌─────────────────┐┌───────┐        
q13328_0: ┤0                ├┤9      ├────────
          │                 ││       │        
q13328_1: ┤1 circuit-133082 ├┤10     ├────────
          │                 ││       │        
q13328_2: ┤2                ├┤11     ├────────
          └─────────────────┘│       │┌──────┐
q13329_0: ───────────────────┤0      ├┤8     ├
                             │       ││      │
q13329_1: ───────────────────┤1      ├┤7     ├
                             │       ││      │
q13329_2: ───────────────────┤2      ├┤6     ├
                             │   QPE ││      │
q13329_3: ───────────────────┤3      ├┤5     ├
                             │       ││      │
q13329_4: ───────────────────┤4      ├┤4     ├
                             │       ││  1/x │
q13329_5: ───────────────────┤5      ├┤3     ├
                             │       ││      │
q13329_6: ───────────────────┤6      ├┤2     ├
                             │       ││      │
q13329_7: ───────────────────┤7      ├┤1     ├
                             │       ││      │
q13329_8: ───────────────────┤8      ├┤0     ├
                             └───────┘│      │
  q13330: ────────────────────────────┤9     ├
                                      └──────┘

In [ ]:
hhl_default.state.data.pop(-1)
state_IBM = Statevector(hhl_default.state).data.real
hhl_default.state.draw()


┌─────────────────┐┌───────┐
q13328_0: ┤0                ├┤9      ├
          │                 ││       │
q13328_1: ┤1 circuit-133082 ├┤10     ├
          │                 ││       │
q13328_2: ┤2                ├┤11     ├
          └─────────────────┘│       │
q13329_0: ───────────────────┤0      ├
                             │       │
q13329_1: ───────────────────┤1      ├
                             │       │
q13329_2: ───────────────────┤2      ├
                             │   QPE │
q13329_3: ───────────────────┤3      ├
                             │       │
q13329_4: ───────────────────┤4      ├
                             │       │
q13329_5: ───────────────────┤5      ├
                             │       │
q13329_6: ───────────────────┤6      ├
                             │       │
q13329_7: ───────────────────┤7      ├
                             │       │
q13329_8: ───────────────────┤8      ├
                             └───────┘
  q13330: ────────────────────────────

In [ ]:
hhl_default.state.data.pop(-1)
state_IBM = Statevector(hhl_default.state).data.real
hhl_default.state.draw()

┌─────────────────┐
q13328_0: ┤0                ├
          │                 │
q13328_1: ┤1 circuit-133082 ├
          │                 │
q13328_2: ┤2                ├
          └─────────────────┘
q13329_0: ───────────────────
                             
q13329_1: ───────────────────
                             
q13329_2: ───────────────────
                             
q13329_3: ───────────────────
                             
q13329_4: ───────────────────
                             
q13329_5: ───────────────────
                             
q13329_6: ───────────────────
                             
q13329_7: ───────────────────
                             
q13329_8: ───────────────────
                             
  q13330: ───────────────────

In [ ]:
qal_hhl.state.data.pop(-1)
qal_hhl.state.data.pop(-1)
qal_hhl.state.draw()



┌──────────────┐┌───────┐
q12_0: ┤0             ├┤9      ├
       │              ││       │
q12_1: ┤1 circuit-127 ├┤10     ├
       │              ││       │
q12_2: ┤2             ├┤11     ├
       └──────────────┘│       │
q13_0: ────────────────┤0      ├
                       │       │
q13_1: ────────────────┤1      ├
                       │       │
q13_2: ────────────────┤2      ├
                       │   QPE │
q13_3: ────────────────┤3      ├
                       │       │
q13_4: ────────────────┤4      ├
                       │       │
q13_5: ────────────────┤5      ├
                       │       │
q13_6: ────────────────┤6      ├
                       │       │
q13_7: ────────────────┤7      ├
                       │       │
q13_8: ────────────────┤8      ├
                       └───────┘
  q14: ─────────────────────────

In [ ]:
qal_hhl.state.data.pop(-1)
qal_hhl.state.draw()


┌──────────────┐
q12_0: ┤0             ├
       │              │
q12_1: ┤1 circuit-127 ├
       │              │
q12_2: ┤2             ├
       └──────────────┘
q13_0: ────────────────
                       
q13_1: ────────────────
                       
q13_2: ────────────────
                       
q13_3: ────────────────
                       
q13_4: ────────────────
                       
q13_5: ────────────────
                       
q13_6: ────────────────
                       
q13_7: ────────────────
                       
q13_8: ────────────────
                       
  q14: ────────────────

In [ ]:
state_qal = Statevector(qal_hhl.state).data.real


In [ ]:
print('state prepared by q_alchemy',state_qal[0:8])
print('state prepared by IBMs default method' ,state_IBM[0:8])
print('Target state',b)

state prepared by q_alchemy [0.41152717 0.12418922 0.42900313 0.40730592 0.53582653 0.26869457
 0.21978396 0.2401433 ]
state prepared by IBMs default method [0.41152717 0.12418922 0.42900313 0.40730592 0.53582653 0.26869457
 0.21978396 0.2401433 ]
Target state [0.41152717 0.12418922 0.42900313 0.40730592 0.53582653 0.26869457
 0.21978396 0.2401433 ]


### Another example with 16 by 16 matrix

In [ ]:
from scipy.sparse import csr_matrix

matrix_size = 16
data = np.random.random(matrix_size)
diag = np.diag(data)
upper_triangle = np.triu(np.random.random((matrix_size, matrix_size)))
upper_triangle = upper_triangle + upper_triangle.conj().T 
np.fill_diagonal(upper_triangle, 0) 

full_matrix = diag + upper_triangle

sparse_matrix = csr_matrix(full_matrix)

A = (sparse_matrix.toarray())
#b = [1, 0, 0, 0, 0, 0, 0, 0]
b = np.random.rand(matrix_size)
b = b/np.linalg.norm(b) #normalize solution


[0.07494194 0.04432369 0.27605236 0.33441072 0.38757454 0.19680435
 0.37378678 0.34879549 0.30437233 0.16353596 0.04761764 0.28538279
 0.17547278 0.00498788 0.11299092 0.3276935 ]


In [ ]:
sp_org = QAlchemyInitialize(b, opt_params={'max_fidelity_loss':0.0})
sp_org.definition.draw(fold=-1)

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
q_0: ┤0                                                                                                                                          ├
     │                                                                                                                                           │
q_1: ┤1                                                                                                                                          ├
     │  LRSP(0.074942,0.044324,0.27605,0.33441,0.38757,0.1968,0.37379,0.3488,0.30437,0.16354,0.047618,0.28538,0.17547,0.0049879,0.11299,0.32769) │
q_2: ┤2                                                                                                                                          ├
     │                                                                                                                                           │
q_3: ┤3                                                                                                                                          ├
     └───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
qc = transpile(sp_org.definition, basis_gates=["id", "rx", "ry", "rz", "cx"])
num_q1 = qc.num_qubits
print('state prepared by Q-alchemy',Statevector(qc).data.real)
print('Target state',b)
qc.draw()

state prepared by Q-alchemy [0.07494194 0.04432369 0.27605236 0.33441072 0.38757454 0.19680435
 0.37378678 0.34879549 0.30437233 0.16353596 0.04761764 0.28538279
 0.17547278 0.00498788 0.11299092 0.3276935 ]
Target state [0.07494194 0.04432369 0.27605236 0.33441072 0.38757454 0.19680435
 0.37378678 0.34879549 0.30437233 0.16353596 0.04761764 0.28538279
 0.17547278 0.00498788 0.11299092 0.3276935 ]


global phase: 2π
     ┌──────────────┐     ┌──────────────┐┌────────┐     ┌────────┐»
q_0: ┤ Ry(0.014093) ├──■──┤ Ry(-0.51323) ├┤ Rz(-π) ├──■──┤ Rz(-π) ├»
     └──────────────┘┌─┴─┐├──────────────┤├────────┤  │  └────────┘»
q_1: ────────────────┤ X ├┤ Ry(-0.39453) ├┤ Rz(-π) ├──┼──────■─────»
                     └───┘└──────────────┘└────────┘┌─┴─┐    │     »
q_2: ───────────────────────────────────────────────┤ X ├────┼─────»
                                                    └───┘  ┌─┴─┐   »
q_3: ──────────────────────────────────────────────────────┤ X ├───»
                                                           └───┘   »
«     ┌─────────────┐                  ┌─────────────┐  ┌────────┐»
«q_0: ┤ Ry(-1.2218) ├───────────────■──┤ Rx(-1.2133) ├──┤ Rz(-π) ├»
«     ├─────────────┴┐┌─────────┐ ┌─┴─┐├─────────────┴┐┌┴────────┤»
«q_1: ┤ Ry(-0.35951) ├┤ Rz(π/2) ├─┤ X ├┤ Rz(-0.61548) ├┤ Ry(π/3) ├»
«     ├─────────────┬┘└─────────┘ └───┘├──────────────┤└┬────────┤»
«q_2: ┤ Ry(-2.7236) ├───────────────■──┤ Rx(-0.50666) ├─┤ Rz(-π) ├»
«     ├─────────────┤ ┌──────────┐┌─┴─┐├──────────────┤┌┴────────┤»
«q_3: ┤ Ry(-2.5978) ├─┤ Rz(-π/2) ├┤ X ├┤ Rz(-0.61548) ├┤ Ry(π/3) ├»
«     └─────────────┘ └──────────┘└───┘└──────────────┘└─────────┘»
«                           ┌─────────┐  ┌──────────────┐                    »
«q_0: ──────────────────■───┤ Rz(π/2) ├──┤ Ry(-0.63359) ├─────────────────■──»
«     ┌──────────────┐┌─┴─┐┌┴─────────┴─┐└─┬──────────┬─┘┌─────────────┐┌─┴─┐»
«q_1: ┤ Rz(-0.61548) ├┤ X ├┤ Rz(2.5261) ├──┤ Ry(2π/3) ├──┤ Rz(-2.5261) ├┤ X ├»
«     └──────────────┘└───┘└┬─────────┬─┘┌─┴──────────┴─┐└─────────────┘└───┘»
«q_2: ──────────────────■───┤ Rz(π/2) ├──┤ Ry(-0.21658) ├─────────────────■──»
«     ┌──────────────┐┌─┴─┐┌┴─────────┴─┐└─┬──────────┬─┘┌─────────────┐┌─┴─┐»
«q_3: ┤ Rz(-0.61548) ├┤ X ├┤ Rz(2.5261) ├──┤ Ry(2π/3) ├──┤ Rz(-2.5261) ├┤ X ├»
«     └──────────────┘└───┘└────────────┘  └──────────┘  └─────────────┘└───┘»
«     ┌─────────┐  ┌───────────┐  
«q_0: ┤ Rz(π/2) ├──┤ Ry(2.542) ├──
«     ├─────────┴┐┌┴───────────┴┐ 
«q_1: ┤ Rz(-π/2) ├┤ Ry(-1.5762) ├─
«     ├─────────┬┘├─────────────┴┐
«q_2: ┤ Rz(π/2) ├─┤ Ry(-0.88113) ├
«     ├─────────┴┐├─────────────┬┘
«q_3: ┤ Rz(-π/2) ├┤ Ry(-2.5826) ├─
«     └──────────┘└─────────────┘

In [ ]:
qal_hhl = HHL().solve(A, qc)
classical_solution = NumPyLinearSolver().solve(A, b)
hhl_default = HHL().solve(A, b) #using IBM's default state preparation method

num_q = qal_hhl.state.num_qubits
x0 = 2**(num_q-1)

qal_sol = get_solution_vector(qal_hhl, x0)
default_sol = get_solution_vector(hhl_default, x0)
classic_sol = classical_solution.state

print('solution with Q-alchemy:', qal_sol)
print('solution with IBM:', default_sol)
print('classical solution:', classic_sol)


solution with Q-alchemy: [ 0.07890896 -0.35581328  0.15805638 -0.01262437  0.38384644 -0.13419981
  0.1125386  -0.04161017]
solution with IBM: [ 0.07890896 -0.35581328  0.15805638 -0.01262437  0.38384644 -0.13419981
  0.1125386  -0.04161017]
classical solution: [ 0.05208701 -0.25248631  0.11220285 -0.02333359  0.27625476 -0.09224288
  0.08055873 -0.02913585  0.22660126 -0.06143259  0.01414967  0.01266434
 -0.10370165 -0.02508739 -0.13648845  0.31475414]
